# Climate Change Analysis and Weather Forecasting

## Task 1: Import Libraries

In [ ]:
import pandas as pd
import seaborn as sns
from prophet import Prophet
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## Task 2: Load Dataset

In [ ]:
# Load dataset into a pandas DataFrame
df = pd.read_csv('/usercode/Dataset.csv')

# Print the DataFrame head
print(df.head())

## Task 3: Convert Date to Datetime Format

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

## Task 4: Get Unique City Names

In [ ]:
cities = df['City'].unique()

## Task 5: Plot Average Daily Maximum and Minimum Temperatures by Cities

In [ ]:
# Iterate over each city
for city in cities:
    # Create a figure and axis object for each city
    fig, ax = plt.subplots(figsize=(10, 5))  # Set the width according to your preference

    # Filter the data for the current city
    city_data = df[df['City'] == city]

    # Calculate the average maximum and minimum temperatures for each year
    avg_max_temps = city_data.groupby(df['Year'])['tmax'].mean()
    avg_min_temps = city_data.groupby(df['Year'])['tmin'].mean()

    # Plot the average maximum and minimum temperatures as lines
    ax.plot(avg_max_temps.index, avg_max_temps, label=f'{city} (Max Temp)')
    ax.plot(avg_min_temps.index, avg_min_temps, label=f'{city} (Min Temp)')

    # Set the labels and title for the graph
    ax.set_xlabel('Year')
    ax.set_ylabel('Temperature (°F)')
    ax.set_title(f'Average Daily Maximum and Minimum Temperatures - {city}')

    # Move the legend outside the graph and prevent it from overlapping
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    # Adjust the layout to make room for the legend
    fig.tight_layout()

    # Show the graph
    plt.show()

## Task 6: Plot Monthly Average Temperatures by Decades

In [ ]:
# Define the decades
decades = range(df['Year'].min() // 10 * 10, df['Year'].max() // 10 * 10 + 1, 10)

# Define the order of months starting from January
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Iterate over each city
for city in cities:
    # Filter the data for the current city
    city_data = df[df['City'] == city].copy()  # Make a copy of the filtered DataFrame

    # Group by decade and month, and calculate the average temperature
    city_data['Decade'] = city_data['Year'] // 10 * 10
    avg_by_decade = city_data.groupby(['Decade', 'Month']).mean(numeric_only=True).reset_index()

    # Pivot the table to reshape the data
    pivot_table = avg_by_decade.pivot(index='Decade', columns='Month', values='tavg')

    # Rearrange the columns
    pivot_table = pivot_table[month_order]

    # Set the size of the heatmap
    plt.figure(figsize=(10, 6))

    # Create the heatmap
    sns.heatmap(pivot_table, cmap='coolwarm', annot=True, fmt=".1f", cbar=True)

    # Set the title and labels
    plt.title(f"Monthly Average Temperatures by Decades - {city}")
    plt.xlabel("Month")
    plt.ylabel("Decade")

    # Show the heatmap
    plt.show()

## Task 7: Plot Average Yearly Precipitation by Cities

In [ ]:
# Group the data by 'City' and 'Year' and calculate the average precipitation for each city and year
average_precipitation = df.groupby(['City', 'Year'])['prcp'].mean()

# Iterate over each city and create a separate plot for each
for city in cities:
    # Filter the data for the current city
    city_data = average_precipitation[city]

    # Create a list of years for the x-axis
    years = city_data.index.tolist()

    # Create a list of average precipitation values for the y-axis
    averages = city_data.tolist()

    # Set the graph size
    plt.figure(figsize=(10, 6))

    # Plot the average yearly precipitation for the current city
    plt.plot(years, averages, marker='o')

    # Set the x-axis label, y-axis label, and plot title
    plt.xlabel('Year')
    plt.ylabel('Average Yearly Precipitation (in)')
    plt.title(f'Average Yearly Precipitation for {city}')

    # Display the plot
    plt.grid(True)
    plt.show()

## Task 8: Plot Temperature and Moving Average by Cities

In [ ]:
# Set the rolling window size for the moving average
rolling_window_size = 30

# Create a separate plot for each city
for city in cities:
    # Filter the data for the specific city
    city_data = df[df["City"] == city].copy()

    # Calculate the moving average of the 'tavg' column
    city_data['tavg_ma'] = city_data['tavg'].rolling(rolling_window_size).mean()

    # Create a plotly figure
    fig = go.Figure()

    # Add traces for the original temperature data and moving average
    fig.add_trace(go.Scatter(x=city_data['Date'], y=city_data['tavg'], mode='lines', name='Original Temperature'))
    fig.add_trace(go.Scatter(x=city_data['Date'], y=city_data['tavg_ma'], mode='lines', name='Moving Average'))

    # Configure the layout
    fig.update_layout(
        title=f'Temperature Data and Moving Average for {city}',
        xaxis_title='Date',
        yaxis_title='Temperature (°F)',
        hovermode='x',
        legend=dict(x=0.7, y=0.9),
        template='plotly_white'
    )

    # Show the interactive plot
    fig.show()

## Task 9: Select and Rename Relevant Columns

In [ ]:
# Select the relevant columns
df = pd.DataFrame(df, columns=['Date', 'tavg', 'City'])

# Rename the columns
df.rename(columns={"Date": "ds", "tavg": "y"}, inplace=True)

## Task 10: Generate Weather Forecasts for Next Five Years

In [ ]:
# Iterate over each city and create a separate graph for each
for city in cities:
    # Filter the data for the current city
    city_data = df[df['City'] == city]

    # Create a Prophet model
    model = Prophet(daily_seasonality=True)

    # Fit the model with the city-specific data
    model.fit(city_data)

    # Generate future dates for prediction (next five years)
    future_dates = model.make_future_dataframe(periods=365*5)

    # Make predictions for the future dates
    forecast = model.predict(future_dates)

    # Set the graph size
    plt.figure(figsize=(10, 6))

    # Plot the overall trend
    model.plot(forecast, xlabel='Year', ylabel='Temperature')

    # Set the plot title
    plt.title(f'Temperature Forecast for {city}')

    # Display the plot
    plt.show()

    # Plot the components: trend, yearly seasonality, and weekly seasonality
    fig = model.plot_components(forecast)